In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from PIL import Image

def custom_img_to_array(img, data_format='channels_last', dtype='float32'):
    """Converts a PIL Image instance to a Numpy array."""
    x = np.asarray(img, dtype=dtype)
    if data_format == 'channels_first':
        if x.ndim == 3:
            x = x.transpose(2, 0, 1)
        elif x.ndim == 4:
            x = x.transpose(0, 3, 1, 2)
    return x

def custom_array_to_img(x, data_format='channels_last', scale=True):
    """Converts a Numpy array to a PIL Image instance."""
    if data_format == 'channels_first':
        x = x.transpose(1, 2, 0)
    if scale:
        x = (x - x.min()) / (x.max() - x.min()) * 255
    return Image.fromarray(x.astype('uint8'))

# Ensure the preview directory exists
save_dir = 'preview'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Initialize the ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Path to the dataset directory
dataset_dir = 'Initial data'

# Iterate through all images in the dataset directory
for filename in os.listdir(dataset_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(dataset_dir, filename)
        img = load_img(img_path)  # This is a PIL image
        x = custom_img_to_array(img) 
        x = x.reshape((1,) + x.shape)
        
        # Generate and save batches of randomly transformed images manually
        i = 0
        for batch in datagen.flow(x, batch_size=1):
            transformed_img = custom_array_to_img(batch[0])
            save_name = f"{os.path.splitext(filename)[0]}_{i}.jpg"
            fname = os.path.join(save_dir, save_name)
            transformed_img.save(fname)
            i += 1
            if i > 100:
                break  # Otherwise, the generator would loop indefinitely


In [1]:
!pip install numpy tensorflow pillow